Copyright (c) Microsoft Corporation. All rights reserved.

## Introduction

This notebook is to make sure that we have all of the dependencies and datasets to train the model.

In this notebook you will
* Install TensorFlow
* Compile the TensorFlow object detection API
* Acquire the Inception V2 pre-trained model.
* Upload both the object detection API and pre-trained model to an Azure ML datastore.


## Install Tensorflow

In [16]:
import sys
import os
import utilities

In [ ]:
!{sys.executable} -m pip install tensorflow
!sudo apt-get install -y python-setuptools

## Compile the Object Detection API

In [ ]:
!git clone https://github.com/tensorflow/models.git
!sudo apt-get install protobuf-compiler python-pil python-lxml python-tk
!{sys.executable} -m pip install Cython
!{sys.executable} -m pip install contextlib2
!{sys.executable} -m pip install jupyter
!{sys.executable} -m pip install matplotlib

In [ ]:
os.chdir("./models/research/")
!wget -O protobuf.zip https://github.com/google/protobuf/releases/download/v3.0.0/protoc-3.0.0-linux-x86_64.zip
!unzip -o protobuf.zip
!./bin/protoc object_detection/protos/*.proto --python_out=.
!bash object_detection/dataset_tools/create_pycocotools_package.sh /tmp/pycocotools
!{sys.executable} -m setup.py sdist
!(cd slim && {sys.executable} -m setup.py sdist)
os.chdir("../../")

In [ ]:
os.mkdir("./packages")
shutil.copy("./models/research/dist/object_detection-0.1.tar.gz","./packages/object_detection-0.1.tar.gz")
shutil.copy("./models/research/slim/dist/slim-0.1.tar.gz","./packages/slim-0.1.tar.gz")
shutil.copy("/tmp/pycocotools/pycocotools-2.0.tar.gz","./packages/pycocotools-2.0.tar.gz")

### Add the Object Detection API to the Datastore

In [ ]:
ws = Workspace.from_config()
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

ds.upload(src_dir="./packages", target_path='./packages',
          overwrite=True, show_progress=True)

### Add the Object Detection API to the local environment

In [ ]:
!{sys.executable} -m pip install ./packages/object_detection-0.1.tar.gz
!{sys.executable} -m pip install ./packages/slim-0.1.tar.gz
!{sys.executable} -m pip install ./packages/pycocotools-2.0.tar.gz

## Acquire and Upload the Inception V2 model

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/mask_rcnn_inception_v2_coco_2018_01_28.tar.gz

In [ ]:
!tar -xzvf mask_rcnn_inception_v2_coco_2018_01_28.tar.gz

In [ ]:
ws = Workspace.from_config()
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

ds.upload(src_dir="./mask_rcnn_inception_v2_coco_2018_01_28", target_path='./mask_rcnn_inception_v2_coco_2018_01_28',
          overwrite=True, show_progress=True)

You can now proceed to next notebook to acquire the dataset.